<a href="https://colab.research.google.com/github/pmoskovi/google-colab/blob/master/FindQueen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uploading files from the local file system

This script allows you to select and upload files from your local file system. This is what I use to upload my training files.

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
import tensorflow as tf


# step 1
filenames = tf.constant(['im-01.jpg', 'im-02.jpg', 'im-03.jpg', 'im-04.jpg', 'im-05.jpg', 'im-06.jpg', 'im-07.jpg', 'im-08.jpg', 'im-09.jpg', 'im-10.jpg', 'im-11.jpg', 
                         'im-51.jpg', 'im-52.jpg', 'im-53.jpg', 'im-54.jpg', 'im-55.jpg', 'im-56.jpg', 'im-57.jpg', 'im-58.jpg', 'im-59.jpg', 'im-60.jpg' ])
labels = tf.constant(
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
0,0,0,0,0,0,0,0,0,0])

# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

dataset = dataset.map(_parse_function)
dataset = dataset.batch(2)

# step 4: create iterator and final input tensor
iterator = dataset.make_one_shot_iterator()
images, labels = iterator.get_next()
